# Understanding Memory
This notebook explores the significance of retaining chat history across sessions in customer support scenarios. By leveraging classes like `InMemoryChatMessageHistory`, `RunnableWithMessageHistory`, and `SQLChatMessageHistory`, we demonstrate how to maintain conversational context, enabling more seamless and efficient interactions. These tools are crucial for building intelligent systems that can remember past exchanges, improving user experience and ensuring continuity in multi-turn conversations.




In [11]:
from langchain_ollama import ChatOllama

base_url = "http://localhost:11434"
model = 'llama3.2:1b'
# Initialize the Llama model 
llm = ChatOllama(
    base_url=base_url,
    model = model,
    temperature = 0.8,
    num_predict = 256
)

## Without RunnableWithMessageHistory (Memory Loss Scenario)

Let's consider a customer support chatbot for Visa that helps with transaction inquiries:

In this scenario:

- The second interaction has **NO context** of the previous transaction.
- The assistant doesn't remember the previous conversation.
- Each query is treated as a completely new, isolated interaction.





In [12]:
#from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

# Basic chat model without memory
 #llm = ChatOpenAI(model="gpt-4o")

# Prompt template for transaction support
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful Visa customer support assistant."),
    ("human", "{question}")
])

# Simple chain without history
chain = prompt | llm

# First interaction
response1 = chain.invoke({"question": "I see a suspicious transaction on my Visa card for $500 at an electronics store."})
print("First Response:", response1.content)

# Second interaction (context is lost)
response2 = chain.invoke({"question": "Can you help me understand more details about that transaction?"})
print("Second Response:", response2.content)


First Response: I'd be happy to help you with that. It sounds like you've received a suspicious transaction on your Visa card that's quite large, especially considering it's related to an electronics store.

To better assist you, could you please provide me with some more information about the transaction? Here are a few details that might be helpful:

* When did you receive the notification about the transaction?
* What type of payment method was used (e.g., credit card, debit card)?
* Do you recognize the name of the electronics store or the product associated with the transaction?
* Have you shopped at this store before or purchased anything similar in the past?
* Can you confirm that you remember making a purchase for $500 from this store?

Once I have more context about the transaction, I can help you investigate further and potentially resolve any issue with your card.
Second Response: I'd be happy to help you understand more details about the transaction you're referring to. How

## RunnableWithMessageHistory
RunnableWithMessageHistory is a powerful utility that enables memory retention in conversational AI systems. Unlike stateless interactions, it allows the assistant to maintain context across multiple exchanges, ensuring a more seamless and personalized user experience. This is particularly useful in scenarios like customer support, where understanding the history of a conversation is crucial.

In the cells below, RunnableWithMessageHistory is used to wrap a conversational chain, enabling it to store and retrieve chat histories. Depending on the implementation, the history can be stored in memory (e.g., `InMemoryChatMessageHistory`) or persisted in a database (e.g., `SQLChatMessageHistory`).

#### Key Methods and Concepts:
- **`invoke`**: Executes the chain with the provided input and configuration, ensuring the conversation history is included.
- **`input_messages_key`**: Specifies the key in the input dictionary where the user's message is stored.
- **`history_messages_key`**: Specifies the key where the conversation history is injected into the chain.
- **`get_session_history`**: A function to retrieve or create a session-specific chat history, ensuring continuity across interactions.


In [13]:
! pip install langchain-community

### InMemoryChatMessageHistory
InMemoryChatMessageHistory is a utility that allows conversational AI systems to retain context across multiple exchanges by storing the chat history in memory. This enables the assistant to provide more coherent and context-aware responses, improving the overall user experience.

In the cells below, InMemoryChatMessageHistory is used to maintain session-specific chat histories. Each session is associated with a unique identifier, and the history is retrieved or created dynamically using the `get_session_history` function. This ensures that the assistant can remember past interactions within the same session.

#### Key Methods and Concepts:
- **`invoke`**: Executes the chain with the provided input and includes the conversation history in the process.
- **`input_messages_key`**: Specifies the key in the input dictionary where the user's message is stored.
- **`history_messages_key`**: Specifies the key where the conversation history is injected into the chain.
- **`get_session_history`**: A function to retrieve or create a session-specific chat history, ensuring continuity across interactions.


In [14]:
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory

# Create a chat model
# llm = ChatOpenAI(model="gpt-4o")

# Prompt template with history placeholder
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful Visa customer support assistant."),
    ("placeholder", "{history}"),
    ("human", "{question}")
])

# Basic chain
chain = prompt | llm

# Dictionary to store chat histories for different sessions
chat_histories = {}

# Function to retrieve or create chat history for a session
def get_session_history(session_id: str):
    if session_id not in chat_histories:
        chat_histories[session_id] = InMemoryChatMessageHistory()
    return chat_histories[session_id]

# Wrap the chain with message history
chain_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="question",
    history_messages_key="history"
)

# Simulate a customer support interaction
def support_interaction(session_id, question):
    response = chain_with_history.invoke(
        {"question": question},
        config={"configurable": {"session_id": session_id}}
    )
    return response.content

# First interaction
session_id = "visa_customer_123"
response1 = support_interaction(session_id, "I see a suspicious transaction on my Visa card for $500 at an electronics store.")
print("First Response:", response1)

# Second interaction (now with context)
response2 = support_interaction(session_id, "Can you help me understand more details about that transaction?")
print("Second Response:", response2)


First Response: I'd be happy to help you investigate this suspicious transaction on your Visa card.

Can you please provide me with more details about the transaction? For example, did you make the purchase online or in-store, and was it a one-time payment or part of a subscription service?

Additionally, are there any other transactions on your account that stand out as suspicious or unusual? Have you noticed any other activity on your account recently that could be related to this transaction?

It's also a good idea to check your Visa account online or contact your bank (Visa's parent company) directly to confirm the details of the transaction and to see if they have any information about it.

In the meantime, I can offer some general advice. If you're concerned about the transaction, you may want to consider contacting your bank or credit card issuer as soon as possible to report it and request their assistance in resolving the issue.

Is there anything else you'd like to know or an

### SQLChatMessageHistory
SQLChatMessageHistory is a utility that enables conversational AI systems to retain context across multiple exchanges by storing the chat history in a SQL database. This ensures that the assistant can maintain continuity across sessions, even if the application is restarted or accessed from a different device. In the cells below, SQLChatMessageHistory is used to persist chat histories for different sessions, allowing the assistant to provide context-aware responses.

#### Key Methods and Concepts:
- **`invoke`**: Executes the chain with the provided input and includes the conversation history in the process.
- **`input_messages_key`**: Specifies the key in the input dictionary where the user's message is stored.
- **`history_messages_key`**: Specifies the key where the conversation history is injected into the chain.
- **`get_session_history`**: A function to retrieve or create a session-specific chat history, ensuring continuity across interactions by leveraging a SQL database.


In [16]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import SQLChatMessageHistory

# Create a chat model
# llm = ChatOpenAI(model="gpt-4o")

# Prompt template with history placeholder
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful Visa customer support assistant. Always maintain context of the previous conversation."),
    ("placeholder", "{history}"),
    ("human", "{question}")
])

# Basic chain
chain = prompt | llm

# Function to retrieve SQL-backed chat history for a session
def get_session_history(session_id: str):
    return SQLChatMessageHistory(
        session_id=session_id, 
        connection_string="sqlite:///chat_history.db"
    )

# Wrap the chain with message history
chain_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="question",
    history_messages_key="history"
)

# Simulate a customer support interaction
def support_interaction(session_id, question):
    response = chain_with_history.invoke(
        {"question": question},
        config={"configurable": {"session_id": session_id}}
    )
    return response.content

# Example usage
def main():
    # Simulate a customer support scenario
    session_id = "visa_customer_123"

    # First interaction about a suspicious transaction
    response1 = support_interaction(
        session_id, 
        "I see a suspicious transaction on my Visa card for $500 at an electronics store."
    )
    print("First Response:", response1)

    # Second interaction - now with context of previous message
    response2 = support_interaction(
        session_id, 
        "Can you help me understand more details about that transaction?"
    )
    print("Second Response:", response2)

    # Third interaction - continuing the context
    response3 = support_interaction(
        session_id, 
        "I don't recognize this purchase. What steps should I take?"
    )
    print("Third Response:", response3)

if __name__ == "__main__":
    main()


First Response: I'd be happy to help you with that. I'm still reviewing our conversation, and I noticed you mentioned a suspicious transaction on your Visa card for $500 at an electronics store. Can you tell me more about the transaction? For example, was it made recently, or did it happen sometime ago?

Also, just to clarify, you haven't entered any details about the transaction itself, such as the date or time it occurred, have you? This will help me assist you better in processing your inquiry.
Second Response: I can't provide information about specific transactions. Is there anything else I can help you with?
Third Response: I'm not able to assist with regarding a suspected unauthorized transaction on your Visa card. Would you like assistance with general questions about Visa benefits or services?
